In [1]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import wordnet as wn

# Download required NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Hardcoded input text
input_text = """
Artificial Intelligence (AI) is transforming the way we live and work. Machine learning, a subset of AI, 
allows computers to learn from data without explicit programming. Deep learning, which is a type of machine learning, 
uses neural networks to process complex patterns. Natural Language Processing (NLP) enables computers to understand 
and interact with human language. These technologies are being applied in various fields including healthcare, 
finance, and transportation.
"""

def extract_keywords(text):
    # Tokenize and tag parts of speech
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)
    
    # Extract nouns and noun phrases as keywords
    keywords = []
    for word, tag in tagged:
        # Looking for nouns and proper nouns
        if tag.startswith(('NN', 'NNP')) and len(word) > 3:
            keywords.append(word.lower())
    
    return list(set(keywords))

def generate_distractors(keyword):
    distractors = set()
    
    # Handle multi-word keywords
    if ' ' in keyword:
        return ['computing', 'technology', 'system']  # Default distractors for compound terms
        
    for syn in wn.synsets(keyword):
        for lemma in syn.lemmas():
            if lemma.name().lower() != keyword.lower():
                distractors.add(lemma.name().lower())
    
    # If we don't have enough distractors, add some defaults
    if len(distractors) < 3:
        default_distractors = ['algorithm', 'software', 'computer']
        distractors.update(default_distractors)
    
    return list(distractors)[:3]

def generate_mcqs(text):
    # 1. Tokenize sentences
    sentences = sent_tokenize(text)
    
    # 2. Extract keywords
    keywords = extract_keywords(text)
    
    # 3. Map keywords to sentences
    mapped_sentences = {}
    for keyword in keywords:
        mapped_sentences[keyword] = [s for s in sentences if keyword.lower() in s.lower()]
    
    # 4. Generate MCQs
    mcqs = []
    for keyword, related_sentences in mapped_sentences.items():
        if related_sentences:  # Only create MCQ if we have a related sentence
            distractors = generate_distractors(keyword)
            
            # Create the question
            question = f"What technology is being described in this sentence: {related_sentences[0]}"
            options = [keyword] + distractors
            
            mcqs.append({
                "question": question,
                "options": options,
                "correct_answer": keyword
            })
    
    return mcqs

# Generate MCQs
mcqs = generate_mcqs(input_text)

# Print the MCQs in a formatted way
for i, mcq in enumerate(mcqs, 1):
    print(f"\nQuestion {i}:")
    print(mcq['question'])
    print("\nOptions:")
    for j, option in enumerate(['A', 'B', 'C', 'D'][:len(mcq['options'])]):
        print(f"{option}) {mcq['options'][j]}")
    print(f"\nCorrect Answer: {mcq['correct_answer']}")
    print("-" * 80)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dsoni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dsoni\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dsoni\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Question 1:
What technology is being described in this sentence: 
Artificial Intelligence (AI) is transforming the way we live and work.

Options:
A) intelligence
B) intelligence_activity
C) intelligence_information
D) intelligence_operation

Correct Answer: intelligence
--------------------------------------------------------------------------------

Question 2:
What technology is being described in this sentence: Deep learning, which is a type of machine learning, 
uses neural networks to process complex patterns.

Options:
A) deep
B) abstruse
C) deeply
D) trench

Correct Answer: deep
--------------------------------------------------------------------------------

Question 3:
What technology is being described in this sentence: Natural Language Processing (NLP) enables computers to understand 
and interact with human language.

Options:
A) language
B) speech
C) oral_communication
D) spoken_communication

Correct Answer: language
-----------------------------------------------------